In [1]:
!nvidia-smi

Fri Dec 29 18:56:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [3]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5310.5/8062.4 GB disk)


In [4]:
def evaluate(log_file_path):
    # Open the file and read its content
    with open(log_file_path, 'r') as file:
        lines = file.readlines()

    # Extract the metrics from the last line of the file
    last_line = lines[-3].strip()
    metrics = last_line.split()

    # Create a dictionary with keys and values
    metrics_dict = {
        "Precision (P)": metrics[3],
        "Recall (R)": metrics[4],
        "mAP50": metrics[5],
        "mAP50-95": metrics[6]
    }
    return metrics_dict

In [5]:
from ultralytics import YOLO
from IPython.display import display, Image

In [6]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="rd4tCrzvJ3lcqGilNLTr")
project = rf.workspace("home-jfi5i").project("football-celebrations")
dataset = project.version(1).download("yolov8")

/kaggle/working/datasets
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
pytoolconfig 1.2.6 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflowjs 4.14.0 

A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Football-Celebrations-1 in yolov8:: 100%|██████████| 1917/1917 [00:00<00:00, 7412.11it/s]


## Custom Training

In [7]:
%cd /kaggle/working/datasets/Football-Celebrations-1
%mkdir valid

/kaggle/working/datasets/Football-Celebrations-1


In [8]:
%cp -r '/kaggle/working/datasets/Football-Celebrations-1/test/images' '/kaggle/working/datasets/Football-Celebrations-1/valid/images'
%cp -r '/kaggle/working/datasets/Football-Celebrations-1/test/labels' '/kaggle/working/datasets/Football-Celebrations-1/valid/labels'

In [9]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True 2>&1 | tee training_logs.txt

/kaggle/working
100%|██████████| 6.23M/6.23M [00:00<00:00, 22.6MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/datasets/Football-Celebrations-1/data.yaml, epochs=50, patience=50, batch=16, imgsz=800, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False

In [10]:
results = evaluate('/kaggle/working/training_logs.txt')
results

{'Precision (P)': '0.987',
 'Recall (R)': '0.988',
 'mAP50': '0.988',
 'mAP50-95': '0.811'}

# Optimizing Using Genetic Algorithm

In [11]:
# model = YOLO('yolov8n.pt')
# model._load('/kaggle/working/runs/detect/train2/weights/best.pt')
# # parameters
# model.model.state_dict()

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from deap import base, creator, tools, algorithms

# Create an instance of the YOLO model
model = YOLO('yolov8n.pt')
model._load('/kaggle/working/runs/detect/train/weights/best.pt')

# Function to evaluate the model with given weights
def evaluate_model(weights):
    # Assign the weights to the model
    assign_weights_to_model(model, weights)
    
    # Implement your evaluation logic here and return a fitness score
    # For example, you can use a validation dataset and calculate accuracy, loss, etc.
    metrics_dict = evaluate('/kaggle/working/training_logs.txt')
    
    # Weighted sum of metrics as the fitness value
    fitness = (
        float(metrics_dict["Precision (P)"]) +
        float(metrics_dict["Recall (R)"]) +
        float(metrics_dict["mAP50"]) +
        float(metrics_dict["mAP50-95"])
    )
    
    return fitness,

# Function to assign weights to the model
def assign_weights_to_model(model, flat_weights):
    start = 0
    for param in model.model.parameters():
        end = start + np.prod(param.shape)
        param.data = torch.from_numpy(flat_weights[start:end].reshape(param.shape)).to(torch.float32)
        start = end


# Function to get the model weights as a flat numpy array
def get_model_weights(model):
    state_dict = model.model.state_dict()
    return np.concatenate([param.cpu().numpy().flatten() for param in state_dict.values()])

# Define the genetic algorithm parameters
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.8
NUM_GENERATIONS = 10

# Define the DEAP creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

# Create the DEAP toolbox
toolbox = base.Toolbox()

# Register the individual (a set of weights) and the population
toolbox.register("individual", tools.initRepeat, creator.Individual, np.random.rand, len(get_model_weights(model)))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the genetic operators
toolbox.register("evaluate", evaluate_model)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=MUTATION_RATE)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create the initial population
population = toolbox.population(n=POPULATION_SIZE)

# Main genetic algorithm loop
for gen in range(NUM_GENERATIONS):
    # Evaluate the entire population
    fitness_values = list(map(toolbox.evaluate, population))
    
    # Assign the fitness values to each individual
    for ind, fit in zip(population, fitness_values):
        ind.fitness.values = fit

    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))

    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation to the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < CROSSOVER_RATE:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if np.random.rand() < MUTATION_RATE:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the offspring
    fitness_values = list(map(toolbox.evaluate, offspring))
    for ind, fit in zip(offspring, fitness_values):
        ind.fitness.values = fit

    # Replace the old population by the offspring
    population[:] = offspring

# Get the best individual in the final population
best_individual = tools.selBest(population, 1)[0]

# Use the best weights for your YOLO model
best_weights = best_individual
assign_weights_to_model(model, best_weights)

# Now 'model' is loaded with the best weights obtained from the genetic algorithm


In [13]:
!yolo task=detect mode=val model='/kaggle/working/runs/detect/train/weights/best.pt' data={dataset.location}/data.yaml epochs=1 imgsz=800 plots=True 2>&1 | tee training_logs.txt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs
val: Scanning /kaggle/working/datasets/Football-Celebrations-1/valid/labels.cache... 85 images, 0 backgrounds, 0 corrupt: 100%|██████████| 85/85 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.83it/s]
                   all         85         85      0.987      0.988      0.988      0.815
Speed: 4.0ms pre-process, 4.3ms inference, 0.0ms loss, 7.7ms post-process per image
